In [1]:
from __future__ import print_function, unicode_literals
from unicodedata import normalize
import library_py2 as lib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from sklearn.metrics import roc
# from sklearn.model_selection import traintestsplit
# from sklearn.preprocessing import 
import json
import warnings
# from nltk.punkt impor
import nltk
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
import string
from nltk.stem.wordnet import WordNetLemmatizer, wordnet
from nltk.stem.porter import PorterStemmer
from nltk.collocations import *
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings(action='once')
%matplotlib inline
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding, Masking
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
import argparse
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical

In [3]:
f = open(os.path.join('glove.6B/glove.6B.{}d.txt'.format(100)))
for line in f:
    values = line.split()
    w = values[0]
    if w=='lovan':
        print(np.asarray(values[1:], dtype='float32'))

[ 0.17929    0.20695   -0.62063   -0.4228    -0.05704   -0.23936
  0.18821   -0.49962   -0.65871   -0.028988  -0.42851   -0.12152
 -0.50426    0.039459  -0.95354    0.35556    0.16659    0.35427
 -0.058108   0.4491    -0.13134    0.34017   -0.2323     0.29535
  0.0031521  0.4095    -0.069262   0.2878    -0.11508    0.46259
  0.18097   -0.34058   -0.081104   0.56952   -0.17003   -0.12939
  0.15523   -0.30052   -0.12025   -0.14236    0.51253    0.013279
  0.034799   0.262     -0.099857  -0.020562  -0.49418   -0.020085
  0.55286    0.19365    0.33676    0.3098     0.050392  -0.7602
  0.15757    0.35251    0.076869  -0.53666   -0.96144   -0.84084
 -0.24946    0.12436    0.42397   -0.067973  -0.21396   -0.43569
  0.39261   -0.24289    0.19704    0.34781   -0.052152   0.22035
 -0.42281    0.43329    0.22818   -0.56003    0.63438   -0.37318
  0.4541     0.14276   -0.71991   -0.10251    0.28526    0.013628
  0.63518   -0.032547   0.022885  -0.45873   -0.39914    0.71629
 -0.11674    0.11912   

In [3]:
from keras.preprocessing.sequence import pad_sequences

In [4]:
data_path='/models'

In [124]:
df = pd.read_pickle('lstm_1.pkl')

In [221]:
df.head()

,answers_lstm,text,text_len,answers_lstm_len,fulltext
0,late 1990s,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
1,sing dance,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
2,2003,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,83,2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
3,houston texas,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
4,late 1990s,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,3,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...


In [222]:
df = df.loc[df.answers_lstm_len>=2]

In [227]:
# df.answers_lstm = df.answers_lstm.apply(lambda x: x[:-1])
df.head()

,answers_lstm,text,text_len,answers_lstm_len,fulltext
0,late 1990s,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
1,sing dance,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
2,2003,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,83,1,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
3,houston texas,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
4,late 1990s,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...


In [226]:
df.answers_lstm_len = df.answers_lstm_len - 1

/Users/flatironschool/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [233]:
train_data[23]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [127]:
df1 = df.loc[df.answers_lstm_len==2]

In [129]:
df1.head()

,answers_lstm,text,text_len,answers_lstm_len,fulltext
2,2003,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,83,2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
11,2003,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,80,2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
12,five,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...,84,2,beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...
15,act,follow disbandment destiny child june 2005 rel...,115,2,follow disbandment destiny child june 2005 rel...
17,six,follow disbandment destiny child june 2005 rel...,114,2,follow disbandment destiny child june 2005 rel...


In [130]:
ndf = df1.sample(frac=1).reset_index(drop=True)


In [131]:
len(ndf)

32629

In [132]:
ndf = ndf[:10000]

In [133]:
len(ndf)

10000

In [144]:
ndf.head()

,answers_lstm,text,text_len,answers_lstm_len,fulltext
0,arabic,lebanon part christian population considers le...,50,2,lebanon part christian population considers le...
1,1668,chamorros guams indigenous people settle islan...,72,2,chamorros guams indigenous people settle islan...
2,sarong,dress westernize clothing jean tshirts somali ...,30,2,dress westernize clothing jean tshirts somali ...
3,archiveitorg,1996 brewster kahle bruce gilliat develop soft...,87,2,1996 brewster kahle bruce gilliat develop soft...
4,mahayana,accord jan natty term mahāyāna great vehicle o...,36,2,accord jan natty term mahāyāna great vehicle o...


In [136]:
ndf.answers_lstm[230]

'weekend '

In [139]:
ndf.answers_lstm = ndf.answers_lstm.apply(lambda x: x[:-1])

In [143]:
ndf.answers_lstm[0]

'arabic'

In [145]:
ndf.answers_lstm_len = ndf.answers_lstm_len.apply(lambda x: x-1)

In [146]:
ndf.head()

,answers_lstm,text,text_len,answers_lstm_len,fulltext
0,arabic,lebanon part christian population considers le...,50,1,lebanon part christian population considers le...
1,1668,chamorros guams indigenous people settle islan...,72,1,chamorros guams indigenous people settle islan...
2,sarong,dress westernize clothing jean tshirts somali ...,30,1,dress westernize clothing jean tshirts somali ...
3,archiveitorg,1996 brewster kahle bruce gilliat develop soft...,87,1,1996 brewster kahle bruce gilliat develop soft...
4,mahayana,accord jan natty term mahāyāna great vehicle o...,36,1,accord jan natty term mahāyāna great vehicle o...


In [228]:
df1 = ndf.loc[ndf.answers_lstm_len == 1]
df2 = ndf.loc[ndf.answers_lstm_len == 2]


In [230]:
df1.head()

,answers_lstm,text,text_len,answers_lstm_len,fulltext
0,arabic,lebanon part christian population considers le...,50,1,lebanon part christian population considers le...
1,1668,chamorros guams indigenous people settle islan...,72,1,chamorros guams indigenous people settle islan...
2,sarong,dress westernize clothing jean tshirts somali ...,30,1,dress westernize clothing jean tshirts somali ...
3,archiveitorg,1996 brewster kahle bruce gilliat develop soft...,87,1,1996 brewster kahle bruce gilliat develop soft...
4,mahayana,accord jan natty term mahāyāna great vehicle o...,36,1,accord jan natty term mahāyāna great vehicle o...


In [231]:
df1.to_pickle('lstm_ans1.pkl')
df2.to_pickle('lstm_ans2.pkl')


In [234]:
def load_data(data):
    # 
    data.reset_index(inplace=True, drop=True)
    
    train_df = data.iloc[:round(0.8*len(data))]
    valid_df = data.iloc[round(0.8*len(data)):round(0.9*len(data))]
    test_df = data.iloc[round(0.9*len(data)):]
    

    # build the complete vocabulary, then convert text data to dict of integer-word pairs
    vocabulary = lib.get_total_vocab(data, column='fulltext')
    train_data = lib.file_to_word_ids(vocabulary, train_df, input_col='text', output_col = 'answers_lstm', 
                                      max_input=data.text_len.max(), max_output=data.answers_lstm_len.max())
    valid_data = lib.file_to_word_ids(vocabulary, valid_df, input_col='text', output_col = 'answers_lstm', 
                                      max_input=data.text_len.max(), max_output=data.answers_lstm_len.max())
    test_data = lib.file_to_word_ids(vocabulary, test_df, input_col='text', output_col = 'answers_lstm', 
                                      max_input=data.text_len.max(), max_output=data.answers_lstm_len.max())
    vocabulary_size = len(vocabulary)
    reversed_dictionary = dict(zip(vocabulary.values(), vocabulary.keys()))

    print("First five train ... ", train_data[:5])
    print("Vocabulary   ", vocabulary)
    print(" Size of vocab .  ", vocabulary_size)
    train_data_lst=train_data.tolist()
    print(" ".join([reversed_dictionary[x] for x in train_data_lst[10]]))
    return train_data, valid_data, test_data, vocabulary, reversed_dictionary, vocabulary_size



In [235]:
train_data, valid_data, test_data, vocabulary, reversed_dictionary, vocabulary_size = load_data(ndf)


First five train ...  [[    0     0     0 ... 37771     1 35194]
 [    0     0     0 ... 38707     1 29945]
 [    0     0     0 ... 46718     1 28003]
 [    0     0     0 ... 45279     1  2025]
 [    0     0     0 ...  2488     1 36592]]
Vocabulary    {'': 1, 'radius': 2, 'edmonds': 3, 'pediatric': 4, '1500s': 5, 'rembrandt': 6, 'skate': 7, 'mythographic': 8, '979': 9, 'frigatebird': 10, 'monocondylia': 11, 'devour': 12, '1203–1283—the': 13, 'tapeworm': 14, 'impend': 15, 'majestic': 16, 'jazz': 17, 'optimistic': 18, 'nasserinspired': 19, 'lancaster': 20, 'colonias': 21, 'detract': 22, 'marijin': 23, 'whether': 24, 'transaction': 25, 'psalm': 26, 'napoloens': 27, 'circumcision': 28, 'inferior': 29, 'raitt': 30, 'councilors': 31, 'tennessee': 32, 'noncinematic': 33, 'messeschnellweg': 34, 'theism': 35, 'soult': 36, 'trapelia': 37, 'remit': 38, 'oppidan': 39, 'raceethnicity': 40, 'kric': 41, 'plea': 42, 'hypochondriasis': 43, 'genre': 44, 'malpighian': 45, 'leader\u200d—\u200cand': 46, 'i

In [240]:
train_data[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [246]:
seq=''
for x in train_data[5]:
    if x!=0:
        seq += reversed_dictionary[x] + ' '
        
print(seq)

three member parliament city royston smith conservative southampton itchen constituency cover east city dr alan whitehead labour southampton test cover west city caroline nokes conservative romsey southampton north include northern portion city  many representative southampton parliament  three 


In [203]:
class KerasBatchGenerator(object):
    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step, answer_length=1):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        self.current_idx = 0
        self.skip_step = skip_step
        self.vocabulary_size = len(vocabulary)
        self.answer_length = answer_length
        
        
    #     num_steps = nr of words that will be fed into the input layer; i.e. the nr of words the model will use to predict the next word
    #     skip_steps = how far to move the window after the prediction is made

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps)) # input nn nodes, same dimensions as batchsize x nr of words in one "window"
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary_size)) # output nodes
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data): # once the ticker goes over the length of data, reset it
                    self.current_idx = 0
                x[i,:] = self.data[self.current_idx][:-self.answer_length]
                
#                 x[i,:] = self.data[j][:-2]
                # input is the segment of data from current index to current index + step-increment
#                 temp_y = self.data[self.current_idx+1 : self.num_steps + self.current_idx + 1]
                temp_y = self.data[self.current_idx][self.answer_length:]
                # converts all the output y into a 1-hot representation
                y[i,:, :] = to_categorical(temp_y, num_classes=self.vocabulary_size)
#                 self.current_idx += self.skip_step
                self.current_idx+=1
            
            
            yield x, y
        
            
        # x dimensions: nr of samples we specify in the batch first, number of w to base predictions on 
        # y has 3 dims: batchsize, nr time steps and also size of vocabulary


In [204]:
ndf.text_len.max()

313

In [213]:
num_steps = ndf.text_len.max()
batch_size = 10


In [214]:
train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps+1)
valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps+1)

In [215]:
hidden_size = 300
use_dropout=True
num_epochs=20


In [216]:
vocabulary['']

1

In [217]:
from keras.activations import sigmoid
from keras.losses import cosine_proximity

In [218]:
model = Sequential([
    layers.Embedding(input_dim=vocabulary_size, output_dim=hidden_size,input_length=num_steps, mask_zero=True),
    layers.LSTM(hidden_size, return_sequences=True),
    layers.LSTM(hidden_size, return_sequences=True),
    layers.LSTM(hidden_size, dropout=0.25, return_sequences=True),
    layers.TimeDistributed(Dense(vocabulary_size)),
    layers.Activation('softmax')
])

In [219]:

optimizer=Adam()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 313, 300)          15380700  
_________________________________________________________________
lstm_40 (LSTM)               (None, 313, 300)          721200    
_________________________________________________________________
lstm_41 (LSTM)               (None, 313, 300)          721200    
_________________________________________________________________
lstm_42 (LSTM)               (None, 313, 300)          721200    
_________________________________________________________________
time_distributed_14 (TimeDis (None, 313, 51269)        15431969  
_________________________________________________________________
activation_13 (Activation)   (None, 313, 51269)        0         
Total params: 32,976,269
Trainable params: 32,976,269
Non-trainable params: 0
________________________________________________________________

In [220]:
model.fit_generator(train_data_generator.generate(), steps_per_epoch = len(train_data)//(batch_size*20), 
                    epochs=num_epochs,
                    validation_data=valid_data_generator.generate(),
                    validation_steps=len(valid_data)//(batch_size))

Epoch 1/20
 9/40 [=====>........................] - ETA: 28:08 - loss: 11.0617 - categorical_accuracy: 0.0862

KeyboardInterrupt: 

In [ ]:
model = Sequential()
model.add(Embedding(vocabulary_size, hidden_size, input_length=num_steps, mask_zero=True))
model.add((Masking(mask_value=0)))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary_size)))
model.add(Activation('softmax'))

In [ ]:

optimizer=Adam()
model.compile(loss='categorical_hinge', optimizer='adam', metrics=['categorical_accuracy'])

print(model.summary())



In [ ]:
model.fit_generator(train_data_generator.generate(), steps_per_epoch = len(train_data)//(batch_size*20), 
                    epochs=num_epochs,
                    validation_data=valid_data_generator.generate(),
                    validation_steps=len(valid_data)//(batch_size))

In [249]:
train_data[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [283]:
d = np.zeros((1,num_steps))
d[0,:] = train_data[100][:-1]


In [284]:
model

In [285]:
d[0].shape

(313,)

In [288]:
predicted_word

1

In [287]:
seq=''
for x in d[0]:
    if x!=0:
        seq += reversed_dictionary[x] + "  "
seq += '\n' + "ANSWER:  " + reversed_dictionary[train_data[100][-1]] + "; "
pred  = model.predict(d)
for j in range(1,5):
    predicted_word = np.argmax(pred[:, num_steps-1, :])  # test whether it needs to be -1, given that we're outputting the final word (the num_steps is equal to the max length of text)
    seq += "PREDICTION:  " 
    seq += reversed_dictionary[predicted_word]
    
print(seq)

communication  observe  within  plant  organism  ie  within  plant  cell  plant  cell  plant  relate  specie  plant  nonplant  organism  especially  root  zone  plant  root  communicate  rhizome  bacteria  fungi  insect  within  soil  interaction  govern  syntactic  pragmatic  semantic  rulescitation  need  possible  decentralize  nervous  system  plant  original  meaning  word  neuron  greek  vegetable  fiber  recent  research  show  microorganism  plant  communication  process  neuronlike  plant  also  communicate  via  volatile  expose  herbivory  attack  behavior  thus  warn  neighbor  plant  parallel  produce  volatile  attract  parasite  attack  herbivore  stress  situation  plant  overwrite  genome  inherit  parent  revert  grand  greatgrandparents  citation  need    part  plant  communicate  rhizome  bacteria  fungi  insect  within  soil    
ANSWER:  root; PREDICTION:  


In [163]:
vocabulary_size

3414

In [164]:
model.input

<tf.Tensor 'embedding_6_input:0' shape=(?, 206) dtype=float32>

In [162]:
model.output

<tf.Tensor 'activation_6/truediv:0' shape=(?, 206, 3414) dtype=float32>

In [ ]:
dummy_iters = 40
example_training_generator = KerasBatchGenerator(train_data, num_steps, 1, vocabulary,
                                                     skip_step=num_steps+1)
print("Training data:")
for i in range(dummy_iters):
    dummy = next(example_training_generator.generate())
num_predict = 110
true_print_out = "Actual words: "
pred_print_out = "Predicted words: "
for i in range(num_predict):
    data = next(example_training_generator.generate())
    prediction = model.predict(data[0])
    predict_word = np.argmax(prediction[:, num_steps-1, :])
    
    true_print_out += reversed_dictionary[train_data[num_steps + dummy_iters + i]] + " "
    pred_print_out += reversed_dictionary[predict_word] + " "
print(true_print_out)
print(pred_print_out)
print(prediction.shape)

In [ ]:

if args.run_opt ==1:
    model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,)
    #                         validation_data=valid_data_generator.generate(),
    #                         validation_steps=len(valid_data)//(batch_size))
    model.save(data_path + "/final_model.hdf5")
elif args.run_opt ==2:
    model = load_model(data_path + "/mo")
